In [1]:
!pip install pdfplumber pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 921.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 25.5 MB/s eta 0:00:00


In [6]:
import os
import pdfplumber
import pandas as pd
import re  # For extracting numbers and invoice details

# Path to your folder with PDFs
pdf_folder_path = '/content/sample_data/Untitled Folder'

# Initialize an empty list to store data from all PDFs
all_data = []

# Function to safely extract numeric values from strings
def extract_number(text):
    # Use regex to extract numeric value from a string
    match = re.search(r'(\d+[\.,]?\d*)', text)
    return match.group(1).replace(',', '') if match else None

# Function to extract the invoice number based on pattern like 'Invoice #: INV-118'
def extract_invoice_number(text):
    match = re.search(r'Invoice #\s*:\s*(\S+)', text)
    return match.group(1) if match else None

# Function to extract the place of supply (handling multiple words)
def extract_place_of_supply(text):
    match = re.search(r'Place of Supply\s*:\s*([\d]+-[A-Za-z\s]+)', text)
    return match.group(1).strip() if match else None

# Loop through all PDF files in the folder
for pdf_filename in os.listdir(pdf_folder_path):
    if pdf_filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder_path, pdf_filename)

        # Initialize the data dictionary for each PDF
        data = {
            'taxable_value': [],
            'sgst_amount': [],
            'cgst_amount': [],
            'igst_amount': [],
            'sgst_rate': [],
            'cgst_rate': [],
            'igst_rate': [],
            'tax_amount': [],
            'tax_rate': [],
            'final_amount': [],
            'invoice_number': [],
            'invoice_date': [],
            'place_of_supply': [],
            'place_of_origin': [],
            'gstin_supplier': [],
            'gstin_recipient': []
        }

        # Open each PDF file using pdfplumber
        with pdfplumber.open(pdf_path) as pdf:
            first_page = pdf.pages[0]
            text = first_page.extract_text()

            # Extract the invoice number using the custom function
            invoice_number = extract_invoice_number(text)
            if invoice_number:
                data['invoice_number'].append(invoice_number)

            # Extract the place of supply using the custom function
            place_of_supply = extract_place_of_supply(text)
            if place_of_supply:
                data['place_of_supply'].append(place_of_supply)

            # Example: Extract data like in the previous example
            if 'Taxable Amount' in text:
                taxable_value = text.split('Taxable Amount')[-1].strip().split('₹')[-1].strip()
                taxable_value = extract_number(taxable_value)  # Extract numeric part only
                if taxable_value:
                    data['taxable_value'].append(float(taxable_value))

            if 'CGST' in text:
                cgst_amount = text.split('CGST')[1].split('₹')[-1].strip().split('\n')[0].strip()
                cgst_amount = extract_number(cgst_amount)  # Extract numeric part only
                if cgst_amount:
                    cgst_rate = text.split('CGST')[1].split('%')[0].strip().split()[-1]
                    data['cgst_amount'].append(float(cgst_amount.replace(',', '')))
                    data['cgst_rate'].append(float(cgst_rate))

            if 'SGST' in text:
                sgst_amount = text.split('SGST')[1].split('₹')[-1].strip().split('\n')[0].strip()
                sgst_amount = extract_number(sgst_amount)  # Extract numeric part only
                if sgst_amount:
                    sgst_rate = text.split('SGST')[1].split('%')[0].strip().split()[-1]
                    data['sgst_amount'].append(float(sgst_amount.replace(',', '')))
                    data['sgst_rate'].append(float(sgst_rate))

            if 'Total ₹' in text:
                final_amount = text.split('Total ₹')[-1].strip().split('\n')[0].strip()
                final_amount = extract_number(final_amount)  # Extract numeric part only
                if final_amount:
                    data['final_amount'].append(float(final_amount.replace(',', '')))

            if 'Invoice Date' in text:
                invoice_date = text.split('Invoice Date:')[-1].split()[0:3]
                data['invoice_date'].append(" ".join(invoice_date))

            # Manually add the place of origin and GSTIN (this will depend on your PDF)
            data['place_of_origin'].append('Shahdol, MADHYA PRADESH')
            data['gstin_supplier'].append('23AADCU2395N1ZY')

        # Add this PDF's data to the list of all PDFs
        all_data.append(data)

# Combine all data into a single DataFrame with empty list handling
df_all = pd.DataFrame([{
    'taxable_value': d['taxable_value'][0] if d['taxable_value'] else None,
    'sgst_amount': d['sgst_amount'][0] if d['sgst_amount'] else None,
    'cgst_amount': d['cgst_amount'][0] if d['cgst_amount'] else None,
    'igst_amount': d['igst_amount'][0] if d['igst_amount'] else None,
    'sgst_rate': d['sgst_rate'][0] if d['sgst_rate'] else None,
    'cgst_rate': d['cgst_rate'][0] if d['cgst_rate'] else None,
    'igst_rate': d['igst_rate'][0] if d['igst_rate'] else None,
    'tax_amount': (d['sgst_amount'][0] if d['sgst_amount'] else 0) + (d['cgst_amount'][0] if d['cgst_amount'] else 0),
    'tax_rate': (d['sgst_rate'][0] if d['sgst_rate'] else 0) + (d['cgst_rate'][0] if d['cgst_rate'] else 0),
    'final_amount': d['final_amount'][0] if d['final_amount'] else None,
    'invoice_number': d['invoice_number'][0] if d['invoice_number'] else None,
    'invoice_date': d['invoice_date'][0] if d['invoice_date'] else None,
    'place_of_supply': d['place_of_supply'][0] if d['place_of_supply'] else None,
    'place_of_origin': d['place_of_origin'][0] if d['place_of_origin'] else None,
    'gstin_supplier': d['gstin_supplier'][0] if d['gstin_supplier'] else None,
    'gstin_recipient': d['gstin_recipient'][0] if d['gstin_recipient'] else None
} for d in all_data])

# Save the combined DataFrame to Excel
output_excel_path = '/content/sample_data/all_invoices_data.xlsx'
df_all.to_excel(output_excel_path, index=False)

print(f"All PDFs processed and data saved to {output_excel_path}")


All PDFs processed and data saved to /content/sample_data/all_invoices_data.xlsx


In [12]:
import os
import pdfplumber
import pandas as pd
import re

# Path to your folder with PDFs
pdf_folder_path = '/content/sample_data/Untitled Folder'

# Initialize an empty list to store data from all PDFs
all_data = []

# Function to safely extract numeric values from strings
def extract_number(text):
    match = re.search(r'(\d+[\.,]?\d*)', text)
    return match.group(1).replace(',', '') if match else None

# Function to extract invoice number based on pattern
def extract_invoice_number(text):
    match = re.search(r'Invoice #\s*:\s*(\S+)', text)
    return match.group(1) if match else None

# Function to extract place of supply with multi-word handling
def extract_place_of_supply(text):
    match = re.search(r'Place of Supply\s*:\s*([\d]+-[A-Za-z\s]+)', text)
    return match.group(1).strip() if match else None

# Function to validate extracted GSTIN (GSTIN format is 15 alphanumeric characters)
def validate_gstin(gstin):
    return bool(re.match(r'\d{2}[A-Z]{5}\d{4}[A-Z]{1}[A-Z\d]{1}[Z]{1}[A-Z\d]{1}', gstin))

# Function to validate invoice number (basic check, this can be customized based on your needs)
def validate_invoice_number(invoice_number):
    return bool(re.match(r'INV-\d+', invoice_number))

# Function to validate the taxable value (ensure it's a valid number)
def validate_taxable_value(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

# Function to determine the overall trust score based on all validation scores
def determine_trust_score(validations):
    return sum(validations) / len(validations)  # Returns a percentage-like score (e.g., 0.8)

# Loop through all PDF files in the folder
for pdf_filename in os.listdir(pdf_folder_path):
    if pdf_filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder_path, pdf_filename)

        # Initialize the data dictionary for each PDF
        data = {
            'taxable_value': [],
            'sgst_amount': [],
            'cgst_amount': [],
            'igst_amount': [],
            'sgst_rate': [],
            'cgst_rate': [],
            'igst_rate': [],
            'tax_amount': [],
            'tax_rate': [],
            'final_amount': [],
            'invoice_number': [],
            'invoice_date': [],
            'place_of_supply': [],
            'place_of_origin': [],
            'gstin_supplier': [],
            'gstin_recipient': [],
            'invoice_number_valid': [],  # Validation score for invoice number
            'taxable_value_valid': [],   # Validation score for taxable value
            'gstin_supplier_valid': [],  # Validation score for GSTIN
            'trust_score': []            # Overall trust score
        }

        # Open each PDF file using pdfplumber
        with pdfplumber.open(pdf_path) as pdf:
            first_page = pdf.pages[0]
            text = first_page.extract_text()

            # Extract the invoice number using the custom function
            invoice_number = extract_invoice_number(text)
            place_of_supply = extract_place_of_supply(text)

            if 'Taxable Amount' in text:
                taxable_value = text.split('Taxable Amount')[-1].strip().split('₹')[-1].strip()
                taxable_value = extract_number(taxable_value)

            gstin_supplier = '23AADCU2395N1ZY'  # Example GSTIN, replace with your extraction method

            # Validate data (Validation Scores: True = 1, False = 0)
            invoice_valid = validate_invoice_number(invoice_number) if invoice_number else False
            taxable_value_valid = validate_taxable_value(taxable_value) if taxable_value else False
            gstin_supplier_valid = validate_gstin(gstin_supplier) if gstin_supplier else False

            # Append individual validation scores to the data
            data['invoice_number_valid'].append(invoice_valid)
            data['taxable_value_valid'].append(taxable_value_valid)
            data['gstin_supplier_valid'].append(gstin_supplier_valid)

            # Calculate trust score based on all validations
            validations = [invoice_valid, taxable_value_valid, gstin_supplier_valid]
            trust_score = determine_trust_score(validations)

            # Append data and trust score
            data['invoice_number'].append(invoice_number)
            data['taxable_value'].append(taxable_value)
            data['place_of_supply'].append(place_of_supply)
            data['gstin_supplier'].append(gstin_supplier)
            data['trust_score'].append(trust_score)  # Add trust score to the data

        # Add this PDF's data to the list of all PDFs
        all_data.append(data)

# Combine all data into a single DataFrame with empty list handling
df_all = pd.DataFrame([{
    'invoice_number': d['invoice_number'][0] if d['invoice_number'] else None,
    'invoice_number_valid': d['invoice_number_valid'][0] if d['invoice_number_valid'] else None,
    'taxable_value': d['taxable_value'][0] if d['taxable_value'] else None,
    'taxable_value_valid': d['taxable_value_valid'][0] if d['taxable_value_valid'] else None,
    'place_of_supply': d['place_of_supply'][0] if d['place_of_supply'] else None,
    'gstin_supplier': d['gstin_supplier'][0] if d['gstin_supplier'] else None,
    'gstin_supplier_valid': d['gstin_supplier_valid'][0] if d['gstin_supplier_valid'] else None,
    'trust_score': d['trust_score'][0] if d['trust_score'] else None  # Add trust score to output
} for d in all_data])

# Save the combined DataFrame to Excel
output_excel_path = '/content/sample_data/all_invoices_data_with_scores.xlsx'
df_all.to_excel(output_excel_path, index=False)

print(f"All PDFs processed and data saved to {output_excel_path}")


All PDFs processed and data saved to /content/sample_data/all_invoices_data_with_scores.xlsx
